# Fix pathing

In [1]:
import sys


sys.path.append("../..")


In [2]:
import constants

import os


constants.PROJECT_DIRECTORY_PATH = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(constants.PROJECT_DIRECTORY_PATH))))


# Imports

In [3]:
import plotter
import datahandler

import matplotlib.pyplot
import numpy as np
import pandas as pd
import seaborn as sns
import IPython.display


# Constants

In [4]:
FOLDER_NAME = "ex_4_least_NONE"

FOLDER_PATH = os.path.join(os.path.dirname(constants.PROJECT_DIRECTORY_PATH), "Simulator", "data", FOLDER_NAME)


# Methods

In [5]:
def load_csv(filename = "events"):
    dataframes = []
    for seed in list(range(10)):
        dataframes.append(pd.read_csv(os.path.join(FOLDER_PATH, filename + "_seed=" + str(seed) + ".csv")))
    
    dataframe = pd.concat(dataframes, ignore_index=True)

    response_time_cols = [
        'duration_incident_creation',
        'duration_resource_appointment',
        'duration_resource_preparing_departure',
        'duration_dispatching_to_scene'
    ]
    dataframe['total_response_time'] = dataframe[response_time_cols].sum(axis=1)

    dataframe["complied"] = np.nan
    # Define the criteria for response times
    criteria = {
        ('A', True): 12 * 60,
        ('A', False): 25 * 60,
        ('H', True): 30 * 60,
        ('H', False): 40 * 60
    }

    for (triage, urban), group in dataframe.groupby(['triage_impression_during_call', 'urban']):
        if triage == 'V1':
            continue
    
        limit = criteria.get((triage, urban))
        if limit is not None:
            dataframe.loc[group.index, 'complied'] = group['total_response_time'] < limit

    return dataframe


In [6]:
def plot_response_time_density(df):
    matplotlib.pyplot.figure(figsize=(12, 8))

    for triage_category in ["A", "H", "V1"]:
        subset = df[df['triage_impression_during_call'] == triage_category]
        sns.kdeplot(subset['total_response_time'] / 60, bw_adjust=0.5, label=f'Triage {triage_category}', fill=True)

    matplotlib.pyplot.title('Response Time Density by Triage Category')
    matplotlib.pyplot.xlabel('Response Time (minutes)')
    matplotlib.pyplot.ylabel('Density')
    matplotlib.pyplot.xlim(left=0)
    matplotlib.pyplot.legend()
    matplotlib.pyplot.show()


In [7]:
def print_table(df: pd.DataFrame):
    # Use a dictionary within agg to specify aggregations for different columns
    stats = df.groupby(['triage_impression_during_call', 'urban']).agg({
        'total_response_time': ['mean', 'median'],
        'complied': 'mean'  # Calculate the mean of the boolean 'complied', which effectively gives the compliance rate
    }).reset_index()

    # Flatten the multi-level column names resulting from aggregation
    stats.columns = ['Triage', 'Urban', 'Mean Response Time', 'Median Response Time', 'Compliance Rate']

    # Convert response times from seconds to minutes and round to 2 decimal places
    stats['Mean Response Time'] = (stats['Mean Response Time'] / 60).round(2)
    stats['Median Response Time'] = (stats['Median Response Time'] / 60).round(2)

    # Convert the compliance rate to percentage and round to 2 decimal places
    # Fill NaN values with 'N/A' for compliance rate
    stats['Compliance Rate'] = stats['Compliance Rate'].apply(lambda x: f"{x*100:.2f}%" if pd.notna(x) else "N/A")

    # Map boolean 'Urban' to 'Yes' or 'No'
    stats['Urban'] = stats['Urban'].map({True: 'Yes', False: 'No'})

    # Sort the DataFrame
    stats.sort_values(by=["Urban", "Triage"], ascending=[False, True], inplace=True)

    # Display the DataFrame using IPython display capabilities, hiding the index
    IPython.display.display(stats.style.hide(axis='index'))


In [8]:
def print_info(dataframe: pd.DataFrame):
    filtered_df = dataframe.dropna(subset=['complied'])

    # compliance
    numerator = filtered_df['complied'].count()
    true_count = filtered_df['complied'].sum()

    compliance_rate = true_count / numerator

    print(f"Compliance Rate: {(abs(compliance_rate - 1) * 100):.2f}")

    # complaince (U)
    # compliance (R)
    compliance_rates = filtered_df.groupby('urban')['complied'].agg(lambda x: x.mean())

    for group, rate in compliance_rates.items():
        print(f"Compliance Rate for {'Urban' if group else 'Non-Urban'}: {(abs(rate - 1) * 100):.2f}")


# Main

In [9]:
dataframe_acc = load_csv(filename="events_ACC")
dataframe_u = load_csv(filename="events_U")
dataframe_pp = load_csv(filename="events_PP")
dataframe_sls = load_csv(filename="events_SLS")
dataframe_ga = load_csv(filename="events_GA")
dataframe_ma = load_csv(filename="events_MA")
dataframe_ouh = load_csv(filename="events_OUH")

dataframe_r = load_csv(filename="events_R")
dataframe_ur = load_csv(filename="events_UR")
dataframe_ip = load_csv(filename="events_IP")


C:\Users\sindr\AppData\Local\Temp\ipykernel_11996\3357299027.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[False  True False False False False  True False False False False  True
 False False False False  True False False False False  True False False
 False False  True False False False False  True False False False False
  True False False False False  True False False False False  True False
 False False]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dataframe.loc[group.index, 'complied'] = group['total_response_time'] < limit
C:\Users\sindr\AppData\Local\Temp\ipykernel_11996\3357299027.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ True  True  True False  True  True  True  True False  True  True  True
  True False  True  True  True  True False  True  True  True  True False
  True

In [13]:
print_info(dataframe_ip)


Compliance Rate: 12.01
Compliance Rate for Non-Urban: 35.45
Compliance Rate for Urban: 10.14
